In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import utils

import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=['Medical_Cost_Prediction.pdf']
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

4 

<class 'llama_index.schema.Document'>
Doc ID: 930ee95d-9cd1-4248-b53b-d1bcf00511bd
Text: 11/28/23, 11:00 PM README.md - Grip localhost:6419 1/4README.md
Medical Cost Prediction Predicting medical costs of individuals based
on different features using several ML (Regression) algorithms. The
application was deployed on AWS EC2 through AWS ECR (Dockerized
Container). Dataset The Medical Cost Prediction consists of around
1300 records a...


In [5]:
# merging all the documents into a single document
from llama_index import Document

document = Document(text='\n\n'.join(doc.text for doc in documents))

## Auto-Merging Retrieval Setup

In [6]:
from utils import build_automerging_index, get_automerging_query_engine
from llama_index.llms import OpenAI

index = build_automerging_index(
    [document],
    llm=OpenAI(model='gpt-3.5-turbo',temperature=0.1),
    save_dir='merging_index'
)

In [7]:
query_engine = get_automerging_query_engine(index, similarity_top_k=10)

## TruLens Evaluation

In [8]:
from trulens_eval import Tru

Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### Two Layers

In [9]:
auto_merging_index_0 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512],
)

In [10]:
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [11]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id ='app_0'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [12]:
questions = []
with open('eval_questions.txt','r') as file:
    for line in file:
        item = line.strip()
        questions.append(item)

In [13]:
questions

['What are the technologies used in the project?',
 'What is the installation process?',
 'Can you talk about the dataset used in the project?',
 'Can you explain the usage of the project?',
 'What is the process of deployment used in the project?']

In [14]:
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [15]:
run_evals(questions, tru_recorder, auto_merging_engine_0)

> Merging 1 nodes into parent node.
> Parent node id: 5cc78f86-481e-4d22-bb0a-95455b676714.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 4/4License
This project is licensed under the ...

> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 2/4Installing
Install the required packages fr...

> Merging 1 nodes into parent node.
> Parent node id: 36573faf-1388-4318-9d26-ebdd6abdaacc.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 3/4 . Create an IAM User - with AmazonEC2Conta...

> Merging 1 nodes into parent node.
> Parent node id: dc183938-748b-4039-9e0b-67a98e6c79e3.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 1/4README.md
Medical Cost Prediction
Predictin...



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 2/4Installing
Install the required packages fr...

> Merging 1 nodes into parent node.
> Parent node id: 36573faf-1388-4318-9d26-ebdd6abdaacc.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 3/4 . Create an IAM User - with AmazonEC2Conta...

> Merging 1 nodes into parent node.
> Parent node id: 5cc78f86-481e-4d22-bb0a-95455b676714.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 4/4License
This project is licensed under the ...

> Merging 1 nodes into parent node.
> Parent node id: dc183938-748b-4039-9e0b-67a98e6c79e3.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 1/4README.md
Medical Cost Prediction
Predictin...

> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - G

In [16]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
app_0,0.35625,1.0,0.233333,5.8,0.001836


In [17]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


python(3878) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.34:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Three Layers


In [18]:
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512,128],
)

In [19]:
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [20]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='app_1'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [21]:
run_evals(questions, tru_recorder, auto_merging_engine_1)

> Merging 1 nodes into parent node.
> Parent node id: 5cc78f86-481e-4d22-bb0a-95455b676714.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 4/4License
This project is licensed under the ...

> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 2/4Installing
Install the required packages fr...

> Merging 1 nodes into parent node.
> Parent node id: 36573faf-1388-4318-9d26-ebdd6abdaacc.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 3/4 . Create an IAM User - with AmazonEC2Conta...

> Merging 1 nodes into parent node.
> Parent node id: dc183938-748b-4039-9e0b-67a98e6c79e3.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 1/4README.md
Medical Cost Prediction
Predictin...



python(3891) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 2/4Installing
Install the required packages fr...

> Merging 1 nodes into parent node.
> Parent node id: 36573faf-1388-4318-9d26-ebdd6abdaacc.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 3/4 . Create an IAM User - with AmazonEC2Conta...

> Merging 1 nodes into parent node.
> Parent node id: 5cc78f86-481e-4d22-bb0a-95455b676714.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 4/4License
This project is licensed under the ...

> Merging 1 nodes into parent node.
> Parent node id: dc183938-748b-4039-9e0b-67a98e6c79e3.
> Parent node text: 11/28/23, 11:00 PM README.md - Grip
localhost:6419 1/4README.md
Medical Cost Prediction
Predictin...

> Merging 1 nodes into parent node.
> Parent node id: 203a9d57-c8df-4df8-a9a0-00668107434c.
> Parent node text: 11/28/23, 11:00 PM README.md - G

In [22]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
app_0,0.40000,1.0,0.596667,5.8,0.001836
app_1,0.36875,1.0,0.666667,5.8,0.001837


In [23]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.4.34:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>